In [ ]:
!pip install --user --upgrade git+https://github.com/huggingface/transformers.git
!pip install --upgrade git+https://github.com/huggingface/accelerate.git
!pip install datasets

In [ ]:
!python --version

In [ ]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git

In [1]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
import os
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, AutoConfig
from transformers import BitsAndBytesConfig
import torch
import copy


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/ctp did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


[2023-07-14 13:09:21,622] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
data_num=-1
max_len = 512
cache_data = 'cache_data/full_instruct_set'
load = 'Salesforce/codet5p-16b'
device = "cuda"
# Training
epochs=1
lr=2e-5
lr_warmup_steps=30
batch_size_per_replica=1
grad_acc_steps=16
local_rank=-1
deepspeed="ds.json"
fp16=True

# Logging and stuff
save_dir="saved_models/full_instruct_set"
log_freq=10
save_freq=500
os.makedirs(save_dir, exist_ok=True)
final_checkpoint_dir = os.path.join(save_dir, "final_checkpoint")

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [4]:
def run_training(model, train_data):
    print(f"Starting main loop")

    training_args = TrainingArguments(
        report_to='tensorboard',
        output_dir=save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',

        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size_per_replica,
        gradient_accumulation_steps=grad_acc_steps,
        learning_rate=lr,
        weight_decay=0.05,
        warmup_steps=lr_warmup_steps,

        logging_dir=save_dir,
        logging_first_step=True,
        logging_steps=log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=2,

        local_rank=local_rank,
        # deepspeed=deepspeed,
#         fp16=fp16,
        bf16 = True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
    )

    trainer.train()

    if local_rank in [0, -1]:
        model.save_pretrained(final_checkpoint_dir)
        print(f'  ==> Finish training and save to {final_checkpoint_dir}')

In [5]:
#for deepspeed
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(load)
config = AutoConfig.from_pretrained(load, trust_remote_code=True, revision="main")
config.decoder_start_token_id = tokenizer.bos_token_id
config.pad_token_id = tokenizer.pad_token_id
#for deepspeed
config.max_position_embeddings = max_len
# print('Model hidden size: ', config.cross_attention_hidden_size)

import math
import numpy as np
def get_model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    model_size = sum([np.prod(p.size()) for p in model_parameters])
    return "{}M".format(round(model_size / 1e+6))

def freeze_decoder_except_xattn_codegen(model):
    print(f'Para before freezing: {model.num_parameters()}, trainable para: {get_model_size(model)}')
    
    for param in model.decoder.parameters():
#         print(param, param.requires_grad)
        param.requires_grad = False

    num_decoder_layers = model.decoder.config.n_layer
    for i in range(num_decoder_layers):
        each_decoder_layer = model.decoder.transformer.h[i]
        if hasattr(each_decoder_layer, 'crossattention'):
            each_decoder_layer.crossattention.to(torch.float32)
            continue
#             for param in each_decoder_layer.crossattention.parameters():
#                 param.requires_grad = True
        if hasattr(each_decoder_layer, 'alpha_xattn'):
            each_decoder_layer.alpha_xattn.to(torch.float32)
#             each_decoder_layer.alpha_xattn.requires_grad = True
            continue
        else:
            param.requires_grad = False
    
    print(f'Para after freezing: {model.num_parameters()}, trainable para: {get_model_size(model)}')

def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

def preprocess_function(examples):
    prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
    source = [prompt_input.format_map({'instruction': instruct, 'input': inp}) if inp != ''
                      else prompt_no_input.format_map({'instruction': instruct})
                      for instruct, inp in zip(examples["instruction"], examples["input"])]
    target = [src + output + tokenizer.eos_token for src, output in zip(source, examples["output"])]

    model_inputs = tokenizer(source, max_length=max_len, padding="max_length", truncation=True)
    labels = tokenizer(target, max_length=max_len, padding="max_length", truncation=True)
    model_inputs["decoder_input_ids"] = copy.deepcopy(labels["input_ids"])

    # changing labels: convert all tokens in the duplicate prefix prompt and the padding part to -100
    eos_token_id = tokenizer.eos_token_id
    for x, y in zip(model_inputs["input_ids"], labels["input_ids"]):
        label_prefix_len = x.index(eos_token_id) if eos_token_id in x else len(x)
        y[:label_prefix_len] = [-100] * label_prefix_len

        if eos_token_id in y:
            pad_len = len(y) - y.index(eos_token_id) - 1
            if pad_len > 0:
                y[y.index(eos_token_id) + 1:] = [-100] * pad_len

    # shift labels to the right as the decoder input and add decoder start token id
    decoder_start_id = tokenizer.eos_token_id
    for z in model_inputs["decoder_input_ids"]:
        z[1:] = z[:-1]
        z[0] = decoder_start_id

    model_inputs["labels"] = copy.deepcopy(labels["input_ids"])
    model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    return model_inputs



def load_tokenize_data():
#     if os.path.exists(cache_data):
#         train_data = load_from_disk(cache_data)
#         print(f'  ==> Loaded {len(train_data)} samples')
# #         res =  convert_size(train_data.size_in_bytes)
# #         print('Dataset Size:', res)
#         return train_data, config
#     else:
        datasets = load_dataset("crumb/Clean-Instruct-440k", split="train")
#         datasets = load_dataset("semeru/text-code-codesummarization", split="validation")
        datasets = datasets.select(range(200000))
        res =  convert_size(datasets.size_in_bytes)
        print('Dataset Size:', res)
        train_data = datasets.map(
            preprocess_function,
            batched=True,
            remove_columns=datasets.column_names,
            num_proc=64,
            load_from_cache_file=False,
        )
        print(f'  ==> Loaded {len(train_data)} samples')
        # train_data.save_to_disk(cache_data)
        # print(f'  ==> Saved to {cache_data}')

        return train_data, config

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


In [8]:
train_data, config = load_tokenize_data()
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Found cached dataset parquet (/home/unnati/.cache/huggingface/datasets/crumb___parquet/crumb--Clean-Instruct-440k-cbf10416ad8c4248/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Dataset Size: 961.89 MB


Map (num_proc=64):   0%|          | 0/200000 [00:00<?, ? examples/s]

  ==> Loaded 200000 samples


In [ ]:


#LORA
from peft import LoraConfig, get_peft_model, TaskType
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj","v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.SEQ_2_SEQ_LM
# )


#QLORA



qlora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["q_proj","v_proj"]
)


if data_num != -1:
    train_data = train_data.select([i for i in range(data_num)])

model = AutoModelForSeq2SeqLM.from_pretrained(load,config=config,trust_remote_code=True,
                                              revision="main",
                                              torch_dtype = torch.bfloat16,
                                              load_in_8bit=True,
                                              low_cpu_mem_usage=True, 
                                              quantization_config=bnb_config)
# freeze_decoder_except_xattn_codegen(model)
print('Model: ', convert_size(model.get_memory_footprint()))
model = get_peft_model(model, qlora_config)
print('PEFT Model: ',convert_size(model.get_memory_footprint()))
print(model.print_trainable_parameters())

print(f"  ==> Loaded model from {load}, model size {model.num_parameters()}")

run_training(model, train_data)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model:  8.41 GB
PEFT Model:  8.42 GB
trainable params: 3,743,744 || all params: 8,434,923,520 || trainable%: 0.04438385233871095
None
  ==> Loaded model from Salesforce/codet5p-16b, model size 8434923520
Starting main loop


/home/unnati/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[W reducer.cpp:1298] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


Step,Training Loss
1,22.171200
10,64.239700
20,23.540800
30,17.837500
40,11.003200
50,26.384600
60,17.587700
70,14.697700
80,8.028100
90,5.007500


In [ ]:
tokenizer.save_pretrained(final_checkpoint_dir)

('saved_models/instruct_set/final_checkpoint/tokenizer_config.json',
 'saved_models/instruct_set/final_checkpoint/special_tokens_map.json',
 'saved_models/instruct_set/final_checkpoint/vocab.json',
 'saved_models/instruct_set/final_checkpoint/merges.txt',
 'saved_models/instruct_set/final_checkpoint/added_tokens.json',
 'saved_models/instruct_set/final_checkpoint/tokenizer.json')

In [ ]:
!pip freeze > urqts.txt

# Inference

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
import time, os

load = "Salesforce/instructcodet5p-16b"
device = "cuda"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}
max_len = 512

import re
def truncate(completion):
    import re
    
    def find_re(string, pattern, start_pos):
        m = pattern.search(string, start_pos)
        return m.start() if m else -1

    terminals = [re.compile(r, re.MULTILINE) for r in [re.escape('<|end|>'),"^'''", '^"""', '\n\n\n']]

    prints = list(re.finditer('^print', completion, re.MULTILINE))
    if len(prints) > 1:
        completion = completion[:prints[1].start()]

    defs = list(re.finditer('^def', completion, re.MULTILINE))
    if len(defs) > 1:
        completion = completion[:defs[1].start()]

    start_pos = 0

    terminals_pos = [pos for pos in [find_re(completion, terminal, start_pos) for terminal in terminals] if pos != -1]
    if len(terminals_pos) > 0:
        return completion[:min(terminals_pos)]
    else:
        return completion  

def preprocess_test_ip(test_ip):
    prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
    instruct, inp = test_ip["instruction"], test_ip["input"]
    if inp!="":
        source = prompt_input.format_map({'instruction': instruct, 'input': inp})
    else:
        source = prompt_no_input.format_map({'instruction': instruct})
    return source

test_ip = {"instruction":"""I want to drop a specific row from a dataset. Provide me all the possible ways to achieve the task.
""", "input":""}

fmt_test_ip = preprocess_test_ip(test_ip)
print(fmt_test_ip)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I want to drop a specific row from a dataset. Provide me all the possible ways to achieve the task.


### Response:


In [7]:
actual_output = """\
"To position the text div from the bottom, you can use the CSS property `top` instead of `bottom`. 
This will position the top part of the div at a certain distance from the top of the parent element. 
To make the div overlap the image, you can also use the CSS property `z-index` to ensure that the text div is positioned on 
top of the image. 
Here's an updated CSS code:
``` #article-txt { min-height: inherit; position: absolute; top: 50%; right: 0; transform: translateY(-50%); z-index: 1; } 
#article-img { display: block; position: relative; z-index: 0; } ``` 
The `top: 50%` positions the top edge of the text div at 50% of the parent element's height. 
The `transform: translateY(-50%)` centers the text div vertically. 
The `z-index: 1` sets the text div to be positioned on top of the image, which has a default `z-index` of 0. 
To make the text div overhang the bottom-right corner of the image, you can adjust the `right` value of the text div to 
control how much of the div sticks out past the right edge of the image: 
``` #article-txt { min-height: inherit; position: absolute; top: 50%; right: -20%; transform: translateY(-50%); z-index: 1; } ``` 
In this example, the text div's right edge is positioned 20% to the left of the right edge of the parent element, which would 
make it overhang the bottom-right corner of the image. Adjust the value as needed to achieve your desired effect"""

In [8]:
print(actual_output)

"To position the text div from the bottom, you can use the CSS property `top` instead of `bottom`. 
This will position the top part of the div at a certain distance from the top of the parent element. 
To make the div overlap the image, you can also use the CSS property `z-index` to ensure that the text div is positioned on 
top of the image. 
Here's an updated CSS code:
``` #article-txt { min-height: inherit; position: absolute; top: 50%; right: 0; transform: translateY(-50%); z-index: 1; } 
#article-img { display: block; position: relative; z-index: 0; } ``` 
The `top: 50%` positions the top edge of the text div at 50% of the parent element's height. 
The `transform: translateY(-50%)` centers the text div vertically. 
The `z-index: 1` sets the text div to be positioned on top of the image, which has a default `z-index` of 0. 
To make the text div overhang the bottom-right corner of the image, you can adjust the `right` value of the text div to 
control how much of the div sticks out 

# Inference without fine tuning

In [2]:
tokenizer = AutoTokenizer.from_pretrained(load)
model = AutoModelForSeq2SeqLM.from_pretrained(load,
                                              torch_dtype=torch.bfloat16,
                                              low_cpu_mem_usage=True,
                                              trust_remote_code=True,
                                              quantization_config=bnb_config)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so


/opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/ctp did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/envs/ctp/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...
[2023-07-14 03:25:48,099] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
def inf_without_ft(test_ip):
    start_time= time.time()
    fmt_test_ip = preprocess_test_ip(test_ip)
    encoding = tokenizer(fmt_test_ip, return_tensors="pt").to(device)
    encoding['decoder_input_ids'] = encoding['input_ids'].clone()
    outputs = model.generate(**encoding, max_length=max_len)
    resp = tokenizer.decode(outputs[0], skip_special_tokens=True)
    stop_time=time.time()
    duration =stop_time - start_time
    return resp, duration

In [5]:
resp, duration =inf_without_ft(test_ip)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [6]:
print(str(duration)+' s')
print(resp[len(fmt_test_ip):])

19.437084436416626 s
:I want to drop a specific row from a dataset. Provide me all the possible ways to achieve the task.


# Inference with fine tuning

In [3]:
from peft import PeftModel
from transformers import BitsAndBytesConfig
import os
import torch

save_dir="saved_models/full_instruct_set"
os.makedirs(save_dir, exist_ok=True)
final_checkpoint_dir = os.path.join(save_dir, "final_checkpoint")
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

max_len = 512
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    load,
    return_dict=True,
    low_cpu_mem_usage=True,
    load_in_8bit=True
    # quantization_config=bnb_config
)
ft_model = PeftModel.from_pretrained(base_model, final_checkpoint_dir)
tokenizer = AutoTokenizer.from_pretrained(final_checkpoint_dir)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
def inf_with_ft(test_ip):
    start_time= time.time()
    fmt_test_ip = preprocess_test_ip(test_ip)
    encoding = tokenizer(fmt_test_ip, return_tensors="pt").to(device)
    encoding['decoder_input_ids'] = encoding['input_ids'].clone()
    outputs = ft_model.generate(**encoding, max_length=max_len)
    resp = tokenizer.decode(outputs[0], skip_special_tokens=True)
    stop_time=time.time()
    duration =stop_time - start_time
    return resp, duration

In [5]:
resp, duration =inf_with_ft(test_ip)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
print(str(duration)+' s')
print(resp[len(fmt_test_ip):])

123.7676203250885 s
:DROP TABLE my_table;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\


In [5]:
print(str(duration)+' s')
print(resp[len(fmt_test_ip):])

123.74796605110168 s
:df.drop(['col1', 'col2'], axis=1)
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6], 'B': [1, 2, 3, 4, 5, 6], 'C': [1, 2, 3, 4, 5, 6]}).set_index('A')

# drop the row that has the value NaN in any column.
df.dropna()
# or, if you want to drop a specific column, use the axis parameter.
df.drop('B', axis=1)
# or, if you want to drop a specific row, use the index parameter.
df.drop(index=0)
# or, if you want to drop a specific column, use the columns parameter.
df.drop(columns=['B', 'C'])
# or, if you want to drop a specific row, use the row parameter.
df.drop(row=0)
# or, if you want to drop a specific value, use the threshold parameter.
df.drop(threshold=1)
# or, if you want to drop a specific value, use the subset parameter.
df.drop(subset=['B', 'C'])
# or, if you want to drop a specific value, use the how parameter.
df.drop(how='any')
# or, if you want to drop a specific value, use the subset parameter.
df.drop(subset=['B', 'C'], how='any')
# or, if you want to drop a s

In [5]:
print(str(duration)+' s')
print(resp[len(fmt_test_ip):])

123.92510867118835 s
:DROP TABLE my_table;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\


In [6]:
print(str(duration)+' s')
print(resp[len(fmt_test_ip):])

71.99185061454773 s
:There are several ways to drop a specific row from a dataset. Here are some examples:

1. Drop a specific row by its index: You can use the `drop()` method to drop a specific row by its index. For example, if you have a dataset with 10 rows and you want to drop the row at index 5, you can use the following code:

```python
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'B': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

df.drop(df.index[5], inplace=True)
```

2. Drop a specific row by its label: You can also use the `drop()` method to drop a specific row by its label. For example, if you have a dataset with 10 rows and you want to drop the row with label 'B', you can use the following code:

```python
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'B': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

df.drop(df['B'], inplace=True)
```


In [8]:
import pandas as pd
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'B': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

df.drop(df.index[5], inplace=True)
df

,A,B
0,1,10
1,2,9
2,3,8
3,4,7
4,5,6
6,7,4
7,8,3
8,9,2
9,10,1


In [10]:
import time
start_time= time.time()
ip = """ How to Implement using recursion and cut-off cycle of the counter (like `for i: = 1 downto N do <operator>`) ?"""

encoding = tokenizer(ip, return_tensors="pt").to(device)
encoding['decoder_input_ids'] = encoding['input_ids'].clone()
outputs = ft_model.generate(**encoding, max_length=1024)
stop_time=time.time()
duration =stop_time - start_time
print(duration)
op = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(ip):]
print(truncate(op))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


303.4228594303131


# In[ ]:


In [11]:
print(op)



# In[ ]:


# Solution

# Import the required libraries
import json

# Create a JSON object
json_object = json.dumps({"value": "aész"})

# Print the JSON object
print(json_object)

# Output:
# > > RES: {"value":"aész"}

# The JSON object is now encoded with unicode values in the UTF-8 special characters, like the `json_encode(array("value", "aész"));`

# You can also use the `json.loads()` function to convert a JSON string to a Python object.

# In[ ]:


# Import the required libraries
import json

# Create a JSON string
json_string = '{"value": "aész"}'

# Convert the JSON string to a Python object
json_object = json.loads(json_string)

# Print the Python object
print(json_object)

# Output:
# > > RES: {"value":"aész"}

# The JSON object is now a Python object with unicode values in the UTF-8 special characters, like the `json_encode(array("value", "aész"));`

# You can also use the `json.dumps()` function to convert a Python object to a JSON string.

# In[ ]:


# Import the required

In [14]:
ft_model =  AutoModelForSeq2SeqLM.from_pretrained(save_dir,local_files_only=True,config=config,trust_remote_code=True,
                                              revision="main", 
                                              low_cpu_mem_usage=True, 
                                              )

OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory saved_models/instruct_set.